In [9]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, SeparableConv2D
from tensorflow.keras.layers import Add, GlobalAveragePooling2D, Dense, Dropout, Multiply
from tensorflow.keras.layers import GroupNormalization, Conv1D, AveragePooling2D, Concatenate, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
import os
from glob import glob
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import time

In [10]:
import pandas as pd
import numpy as np
import cv2

def preprocess_image(image):
    img = cv2.resize(image, (227, 227))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    return img

def load_ckplus_from_csv(csv_path):
    df = pd.read_csv(csv_path, on_bad_lines='skip')  # Skip malformed lines
    images, labels = [], []
    class_map = {0: 'anger', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happiness', 5: 'sadness', 6: 'surprise'}
    
    for _, row in df.iterrows():
        try:
            pixels = np.array(row['pixels'].split(), dtype=np.uint8).reshape(48, 48)
            images.append(preprocess_image(pixels))
            labels.append(int(row['emotion']))
        except (ValueError, IndexError) as e:
            print(f"Skipping invalid row due to error: {e}")
            continue

    images = np.array(images)
    labels = np.array(labels)
    print(f"Loaded {len(images)} CK+ images with 7 classes")
    return images, labels

# Load CK+
ck_path = "/Users/adicadi/.cache/kagglehub/datasets/davilsena/ckdataset/versions/2/ckextended.csv"
ck_images, ck_labels = load_ckplus_from_csv(ck_path)

Loaded 920 CK+ images with 7 classes


In [11]:
import os
from glob import glob

def load_kmu_fed(folder_path):
    images, labels = [], []
    emotion_map = {
        'AN': 0,  # Anger
        'DI': 1,  # Disgust
        'FE': 2,  # Fear
        'HA': 3,  # Happiness
        'SA': 4,  # Sadness
        'SU': 5   # Surprise
    }
    
    extensions = ('*.jpg', '*.jpeg', '*.png')
    img_files = []
    for ext in extensions:
        img_files.extend(glob(os.path.join(folder_path, ext)))
    
    if not img_files:
        print(f"No image files found in {folder_path}. Check dataset structure.")
        return np.array(images), np.array(labels)

    print(f"Found {len(img_files)} images.")
    for img_path in img_files:
        filename = os.path.basename(img_path)
        emotion_code = filename.split('_')[1]  # Extract emotion (e.g., 'AN' from '12_AN_gu_185.jpg')
        if emotion_code in emotion_map:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(preprocess_image(img))
                labels.append(emotion_map[emotion_code])
        else:
            print(f"Unknown emotion code {emotion_code} in {filename}")

    images = np.array(images)
    labels = np.array(labels)
    print(f"Loaded {len(images)} KMU-FED images with 6 classes (expected 1106)")
    if len(images) < 1106:
        print("Warning: Missing images. Dataset may be incomplete or partially downloaded.")
    return images, labels

# Load KMU-FED
kmu_path = "/Users/adicadi/.cache/kagglehub/datasets/anandpanajkar/kmu-fed/versions/1"
kmu_images, kmu_labels = load_kmu_fed(kmu_path)

Found 1106 images.
Loaded 1106 KMU-FED images with 6 classes (expected 1106)


In [12]:
# Preprocessing function
def preprocess_image(image):
    img = cv2.resize(image, (227, 227))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=-1)
    return img

# Load CK+ dataset (using 920 images)
def load_ckplus_from_csv(csv_path):
    df = pd.read_csv(csv_path, on_bad_lines='warn')
    images, labels = [], []
    skipped_rows = 0
    class_map = {0: 'anger', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happiness', 5: 'sadness', 6: 'surprise'}
    
    for idx, row in df.iterrows():
        try:
            pixels = np.array(row['pixels'].split(), dtype=np.uint8).reshape(48, 48)
            images.append(preprocess_image(pixels))
            labels.append(int(row['emotion']))
        except (ValueError, IndexError) as e:
            print(f"Row {idx} skipped due to error: {e} | Data: {row}")
            skipped_rows += 1
            continue

    images = np.array(images)
    labels = np.array(labels)
    print(f"Loaded {len(images)} CK+ images with 7 classes (using 920)")
    print(f"Skipped {skipped_rows} rows due to errors.")
    return images, labels

# Load KMU-FED dataset
def load_kmu_fed(folder_path):
    images, labels = [], []
    emotion_map = {
        'AN': 0,  # Anger
        'DI': 1,  # Disgust
        'FE': 2,  # Fear
        'HA': 3,  # Happiness
        'SA': 4,  # Sadness
        'SU': 5   # Surprise
    }
    
    extensions = ('*.jpg', '*.jpeg', '*.png')
    img_files = []
    for ext in extensions:
        img_files.extend(glob(os.path.join(folder_path, ext)))
    
    if not img_files:
        print(f"No image files found in {folder_path}. Check dataset structure.")
        return np.array(images), np.array(labels)

    print(f"Found {len(img_files)} images.")
    for img_path in img_files:
        filename = os.path.basename(img_path)
        emotion_code = filename.split('_')[1]
        if emotion_code in emotion_map:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(preprocess_image(img))
                labels.append(emotion_map[emotion_code])
        else:
            print(f"Unknown emotion code {emotion_code} in {filename}")

    images = np.array(images)
    labels = np.array(labels)
    print(f"Loaded {len(images)} KMU-FED images with 6 classes (expected 1106)")
    if len(images) < 1106:
        print("Warning: Missing images. Dataset may be incomplete or partially downloaded.")
    return images, labels

# Load datasets
ck_path = "/Users/adicadi/.cache/kagglehub/datasets/davilsena/ckdataset/versions/2/ckextended.csv"
kmu_path = "/Users/adicadi/.cache/kagglehub/datasets/anandpanajkar/kmu-fed/versions/1"

ck_images, ck_labels = load_ckplus_from_csv(ck_path)
kmu_images, kmu_labels = load_kmu_fed(kmu_path)

Loaded 920 CK+ images with 7 classes (using 920)
Skipped 0 rows due to errors.
Found 1106 images.
Loaded 1106 KMU-FED images with 6 classes (expected 1106)


In [13]:
# Data augmentation
datagen = ImageDataGenerator(
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Hybrid Channel Attention
def hybrid_channel_attention(input_tensor, reduction_ratio=16):
    channels = input_tensor.shape[-1]
    x = tf.keras.layers.GlobalAveragePooling2D()(input_tensor)
    x = tf.keras.layers.Dense(channels // reduction_ratio, activation='relu')(x)
    x = tf.keras.layers.Dense(channels, activation='sigmoid')(x)
    x = tf.keras.layers.Multiply()([input_tensor, x])
    return x

# Custom Coordinate Space Attention Layer
class CoordinateSpaceAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CoordinateSpaceAttention, self).__init__(**kwargs)
        self.conv1d_x = tf.keras.layers.Conv1D(1, kernel_size=1, padding='same')
        self.conv1d_y = tf.keras.layers.Conv1D(1, kernel_size=1, padding='same')
        self.sigmoid = tf.keras.layers.Activation('sigmoid')

    def call(self, input_tensor):
        batch_size, height, width, channels = input_tensor.shape
        
        # Pool along width to get height-wise attention
        x = tf.keras.layers.AveragePooling2D(pool_size=(1, width))(input_tensor)
        x = tf.squeeze(x, axis=2)  # Shape: (batch_size, height, channels)
        x = self.conv1d_x(x)  # Shape: (batch_size, height, 1)
        x = self.sigmoid(x)  # Shape: (batch_size, height, 1)
        x = tf.expand_dims(x, axis=2)  # Shape: (batch_size, height, 1, 1)
        x = tf.tile(x, [1, 1, width, 1])  # Expand to match width
        
        # Pool along height to get width-wise attention
        y = tf.keras.layers.AveragePooling2D(pool_size=(height, 1))(input_tensor)
        y = tf.squeeze(y, axis=1)  # Shape: (batch_size, width, channels)
        y = self.conv1d_y(y)  # Shape: (batch_size, width, 1)
        y = self.sigmoid(y)  # Shape: (batch_size, width, 1)
        y = tf.expand_dims(y, axis=1)  # Shape: (batch_size, 1, width, 1)
        y = tf.tile(y, [1, height, 1, 1])  # Expand to match height
        
        # Combine and apply attention
        attention = x * y  # Element-wise multiplication
        attention = tf.reduce_mean(attention, axis=-1, keepdims=True)  # Combine into a single attention map
        attention = self.sigmoid(attention)  # Normalize to [0, 1]
        
        return tf.keras.layers.Multiply()([input_tensor, attention])

# Depthwise Attention Convolution (DAC) Block
def dac_block(input_tensor, filters, kernel_size=3):
    x = tf.keras.layers.SeparableConv2D(filters, kernel_size, padding='same')(input_tensor)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = hybrid_channel_attention(x)
    x = CoordinateSpaceAttention()(x)
    return x

# DALDL Model
def build_daldl(num_classes):
    inputs = tf.keras.layers.Input(shape=(227, 227, 1))
    x = tf.keras.layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.GroupNormalization(groups=8)(x)
    
    # Residual blocks with DAC
    for _ in range(3):
        shortcut = x
        x = dac_block(x, 64)
        x = tf.keras.layers.Add()([shortcut, x])
        x = tf.keras.layers.ReLU()(x)
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.models.Model(inputs, x)
    return model

# Build models
daldl_ck = build_daldl(num_classes=7)
daldl_kmu = build_daldl(num_classes=6)

# Compile models
daldl_ck.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, weight_decay=0.0005),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
daldl_kmu.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, weight_decay=0.0005),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
# Single cell for training and evaluation with recommended changes
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score

# Define model with pre-trained ResNet50
def build_model(num_classes):
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(227, 227, 3))
    base_model.trainable = False
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Custom augmentation function
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    scale = tf.random.uniform([], 0.9, 1.1, dtype=tf.float32)
    new_height = tf.cast(227 * scale, tf.int32)
    new_width = tf.cast(227 * scale, tf.int32)
    image = tf.image.resize(image, [new_height, new_width])
    image = tf.image.resize_with_crop_or_pad(image, 227, 227)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, label

# Training and evaluation function
def train_and_evaluate(model, images, labels, num_classes, dataset_name='ck+'):
    k_folds = 5
    batch_size = 64
    epochs = 100
    
    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
    class_weight_dict = dict(enumerate(class_weights))

    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    accuracies, f1_scores = [], []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(images)):
        print(f'Fold {fold + 1}/{k_folds} for {dataset_name}')
        train_images, val_images = images[train_idx], images[val_idx]
        train_labels, val_labels = labels[train_idx], labels[val_idx]

        train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
        train_dataset = train_dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
        train_dataset = train_dataset.shuffle(buffer_size=len(train_images)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

        # Compile model with adaptive learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        # Callbacks
        callbacks = [
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        ]

        model.fit(
            train_dataset,
            epochs=epochs,
            validation_data=val_dataset,
            class_weight=class_weight_dict,
            verbose=1,
            callbacks=callbacks
        )

        val_preds = model.predict(val_images)
        val_preds = np.argmax(val_preds, axis=1)
        accuracies.append(accuracy_score(val_labels, val_preds))
        f1_scores.append(f1_score(val_labels, val_preds, average='weighted'))

    print(f'{dataset_name} - Average Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}')
    print(f'{dataset_name} - Average F1-Score: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}')

# Build and train models
daldl_ck = build_model(num_classes=7)
daldl_kmu = build_model(num_classes=6)

train_and_evaluate(daldl_ck, ck_images, ck_labels, num_classes=7, dataset_name='CK+')
train_and_evaluate(daldl_kmu, kmu_images, kmu_labels, num_classes=6, dataset_name='KMU-FED')

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
Fold 1/5 for CK+
Epoch 1/100


ValueError: Exception encountered when calling Functional.call().

[1mInput 0 of layer "conv1_conv" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 233, 233, 1)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None, 227, 227, 1), dtype=float32)
  • training=True
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [ ]:
# Inference speed test
def measure_inference_time(model, images, num_runs=100):
    start_time = time.time()
    for _ in range(num_runs):
        model.predict(images[:1], verbose=0)
    avg_time = (time.time() - start_time) / num_runs * 1000
    print(f'Average inference time: {avg_time:.2f} ms per image')

measure_inference_time(daldl_ck, ck_images)
measure_inference_time(daldl_kmu, kmu_images)

In [31]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from PIL import Image
import kagglehub
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score

# Download datasets
ck_path = kagglehub.dataset_download("davilsena/ckdataset")
kmu_path = kagglehub.dataset_download("anandpanajkar/kmu-fed")
print("Path to CK+ dataset files:", ck_path)
print("Path to KMU-FED dataset files:", kmu_path)

# Debug: Print directory contents
print("CK+ directory contents:", os.listdir(ck_path))
print("KMU-FED directory contents:", os.listdir(kmu_path))

# Load CK+ dataset from CSV with pixel data
def load_ck_dataset(data_path):
    csv_file = next((os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.csv')), None)
    if not csv_file:
        raise ValueError(f"No CSV file found in {data_path}")

    df = pd.read_csv(csv_file)
    print(f"CSV columns in {csv_file}: {df.columns.tolist()}")

    images = []
    labels = []
    for index, row in df.iterrows():
        pixel_str = row['pixels'].split()
        pixel_values = np.array([int(p) for p in pixel_str], dtype=np.uint8)
        img_size = int(np.sqrt(len(pixel_values)))  # Assuming square image
        if img_size * img_size != len(pixel_values):
            raise ValueError(f"Pixel data for row {index} cannot be reshaped into a square image")
        img = pixel_values.reshape(img_size, img_size, 1)
        img = tf.image.resize(img, [227, 227])
        img = tf.image.grayscale_to_rgb(img)
        img = img / 255.0
        images.append(img)
        labels.append(int(row['emotion']))

    images = np.array(images)
    labels = np.array(labels)
    print(f"Loaded {len(images)} images from {data_path}")
    return images, labels

# Load KMU-FED dataset from image files
def load_kmu_dataset(data_path):
    images = []
    labels = []
    label_map = {'AN': 0, 'DI': 1, 'FE': 2, 'HA': 3, 'SA': 4, 'SU': 5}  # 6 classes for KMU-FED
    
    for filename in os.listdir(data_path):
        if filename.endswith('.jpg'):
            emotion = filename.split('_')[1]
            if emotion in label_map:
                img_path = os.path.join(data_path, filename)
                img = Image.open(img_path).convert('L')  # Convert to grayscale
                img_array = np.array(img)
                height, width = img.size  # Get actual dimensions
                img_array = img_array.reshape(height, width, 1)  # Reshape to original size with channel
                img = tf.image.resize(img_array, [227, 227])  # Resize to model input
                img = tf.image.grayscale_to_rgb(img)  # Convert to RGB
                img = img / 255.0  # Normalize
                images.append(img)
                labels.append(label_map[emotion])
    
    images = np.array(images)
    labels = np.array(labels)
    print(f"Loaded {len(images)} images from {data_path} with shape {images.shape[1:]}")
    return images, labels
# Custom layer for Hybrid Channel Attention (HCA)
class HCALayer(tf.keras.layers.Layer):
    def __init__(self, max_groups=32, **kwargs):
        super(HCALayer, self).__init__(**kwargs)
        self.max_groups = max_groups
        self.group_norm = None
        self.conv1d = None

    def build(self, input_shape):
        channels = input_shape[-1]
        num_groups = min(channels, self.max_groups)
        self.group_norm = tf.keras.layers.GroupNormalization(groups=num_groups)
        self.conv1d = tf.keras.layers.Conv1D(1, kernel_size=3, padding='same', activation='sigmoid')
        super().build(input_shape)

    def call(self, inputs):
        x = self.group_norm(inputs)
        x_mean = tf.reduce_mean(x, axis=[1, 2], keepdims=True)
        x_mean = tf.squeeze(x_mean, axis=[1, 2])
        x_mean = tf.expand_dims(x_mean, axis=1)
        x_att = self.conv1d(x_mean)
        x_att = tf.repeat(x_att, inputs.shape[1] * inputs.shape[2], axis=1)
        x_att = tf.reshape(x_att, [-1, inputs.shape[1], inputs.shape[2], 1])
        return inputs * x_att

    def compute_output_shape(self, input_shape):
        return input_shape

# Custom layer for Coordinate Space Attention (CSA)
class CSALayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CSALayer, self).__init__(**kwargs)
        self.conv = tf.keras.layers.Conv2D(1, (3, 3), padding='same', activation='sigmoid')

    def call(self, inputs):
        h_pool = tf.reduce_mean(inputs, axis=2, keepdims=True)
        w_pool = tf.reduce_mean(inputs, axis=1, keepdims=True)
        h_pool = tf.reshape(h_pool, [-1, 1, 1, inputs.shape[-1]])
        w_pool = tf.reshape(w_pool, [-1, 1, 1, inputs.shape[-1]])
        attn = tf.keras.layers.Concatenate(axis=-1)([h_pool, w_pool])
        attn = self.conv(attn)
        attn = tf.repeat(tf.repeat(attn, inputs.shape[1], axis=1), inputs.shape[2], axis=2)
        return inputs * attn

    def compute_output_shape(self, input_shape):
        return input_shape

# SqueezeNext bottleneck block
def squeeze_next_block(x, filters, strides=1):
    x_shortcut = x
    x = tf.keras.layers.Conv2D(filters // 2, (1, 1), activation='relu6', padding='same')(x)
    x = tf.keras.layers.DepthwiseConv2D((3, 3), strides=strides, padding='same', activation='relu6')(x)
    x = tf.keras.layers.Conv2D(filters // 2, (1, 3), padding='same', activation='relu6')(x)
    x = tf.keras.layers.Conv2D(filters // 2, (3, 1), padding='same', activation='relu6')(x)
    x = HCALayer(max_groups=32)(x)
    x = CSALayer()(x)
    x = tf.keras.layers.Conv2D(filters, (1, 1), activation='relu6', padding='same')(x)
    if strides != 1 or x_shortcut.shape[-1] != filters:
        x_shortcut = tf.keras.layers.Conv2D(filters, (1, 1), strides=strides, padding='same')(x_shortcut)
    x = tf.keras.layers.Add()([x, x_shortcut])
    return x

# Define DALDL model with SqueezeNext backbone
def build_daldl_model(num_classes):
    inputs = tf.keras.layers.Input(shape=(227, 227, 3))
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu6', padding='same')(inputs)
    x = squeeze_next_block(x, filters=32)
    x = squeeze_next_block(x, filters=64, strides=2)
    x = squeeze_next_block(x, filters=128, strides=2)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation='relu6')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# Training and evaluation function
def train_and_evaluate(model, images, labels, num_classes, dataset_name):
    k_folds = 5
    batch_size = 64
    epochs = 100
    
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    accuracies, f1_scores = [], []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(images)):
        print(f'Fold {fold + 1}/{k_folds} for {dataset_name}')
        train_images, val_images = images[train_idx], images[val_idx]
        train_labels, val_labels = labels[train_idx], labels[val_idx]
        
        train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
        train_dataset = train_dataset.shuffle(buffer_size=len(train_images)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])
        
        model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, verbose=1)
        
        val_preds = model.predict(val_images)
        val_preds = np.argmax(val_preds, axis=1)
        accuracies.append(accuracy_score(val_labels, val_preds))
        f1_scores.append(f1_score(val_labels, val_preds, average='weighted'))
    
    print(f'{dataset_name} - Average Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}')
    print(f'{dataset_name} - Average F1-Score: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}')

# Load datasets
ck_images, ck_labels = load_ck_dataset(ck_path)
kmu_images, kmu_labels = load_kmu_dataset(kmu_path)

# Build and train models
daldl_ck = build_daldl_model(num_classes=7)  # 7 classes for CK+
daldl_kmu = build_daldl_model(num_classes=6)  # 6 classes for KMU-FED

train_and_evaluate(daldl_ck, ck_images, ck_labels, num_classes=7, dataset_name='CK+')
train_and_evaluate(daldl_kmu, kmu_images, kmu_labels, num_classes=6, dataset_name='KMU-FED')

Path to CK+ dataset files: /Users/adicadi/.cache/kagglehub/datasets/davilsena/ckdataset/versions/2
Path to KMU-FED dataset files: /Users/adicadi/.cache/kagglehub/datasets/anandpanajkar/kmu-fed/versions/1
CK+ directory contents: ['ckextended.csv']
KMU-FED directory contents: ['12_AN_gu_185.jpg', '04_FE_s03_079.jpg', '11_DI_ej_111.jpg', '02_SU_s01_037.jpg', '02_SU_s01_023.jpg', '11_DI_ej_105.jpg', '05_SU_uj_076.jpg', '01_DI_mr_012.jpg', '03_FE_s02_043.jpg', '01_DI_mr_006.jpg', '03_FE_s02_057.jpg', '05_SU_uj_062.jpg', '05_HA_uj_086.jpg', '04_SU_s03_069.jpg', '11_SA_ej_147.jpg', '02_FE_s01_027.jpg', '06_SA_dy_089.jpg', '02_FE_s01_033.jpg', '03_SU_s02_053.jpg', '06_SA_dy_076.jpg', '03_SU_s02_047.jpg', '08_SU_nh_095.jpg', '05_HA_uj_079.jpg', '07_AN_sw_098.jpg', '08_SU_nh_081.jpg', '07_SA_sw_090.jpg', '07_SA_sw_084.jpg', '05_DI_uj_045.jpg', '01_SU_mr_009.jpg', '12_DI_gu_124.jpg', '02_HA_s01_023.jpg', '12_DI_gu_130.jpg', '11_AN_ej_167.jpg', '11_FE_ej_188.jpg', '11_SU_ej_136.jpg', '12_SA_gu_166

2025-06-14 19:58:57.928346: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [14528,227,227,1] vs. [64,227,227,16]
	 [[{{function_node __inference_one_step_on_data_130891}}{{node functional_8_1/csa_layer_9_1/mul}}]]
2025-06-14 19:58:57.929069: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 8973604330050715878
2025-06-14 19:58:57.929178: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [14528,227,227,1] vs. [64,227,227,16]
	 [[{{function_node __inference_one_step_on_data_130891}}{{node functional_8_1/csa_layer_9_1/mul}}]]
	 [[StatefulPartitionedCall/functional_8_1/dense_37_1/Softmax/_336]]
2025-06-14 19:58:57.929340: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5700340834337074411


InvalidArgumentError: Graph execution error:

Detected at node functional_8_1/csa_layer_9_1/mul defined at (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/adicadi/Desktop/BTUBooks/ResearchModule/Research_Implementation/.venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/adicadi/Desktop/BTUBooks/ResearchModule/Research_Implementation/.venv/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/adicadi/Desktop/BTUBooks/ResearchModule/Research_Implementation/.venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/adicadi/Desktop/BTUBooks/ResearchModule/Research_